<a href="https://colab.research.google.com/github/githubpradeep/notebooks/blob/main/activation_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence_transformers transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 90.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.5 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=50fba5a572f4a8b12ef1ea4467958c16b6b09376723a4e235b67042dee871b6c
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("roneneldan/TinyStories-33M").to("cuda")
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")


In [ ]:
prompt = "Once upon a time there was"

model.to("cuda")
model.eval()

input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda")

output = model.generate(input_ids, max_length=500, do_sample=True)

output_text = tokenizer.decode(output[0], skip_special_tokens=True)
print()
print(output_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Once upon a time there was a little girl named Annie. She had a favorite sock that she always wore. It was the softest, most comfortable sock ever with her all the colors and textures. One day, at night, Annie noticed the sock had vanished from her drawer! She looked everywhere for it, but it wasn't there. Annie was very sad so she asked her mom for help. Her mom suggested she look in the park, maybe they could find it. Together, they searched up and down and in all the things they did. Suddenly, the sock came flying through the air! Annie was so glad, she hugged her mom and thanked her for helping to get the sock back. From that night on, Annie always made sure to keep her sock safe and sound!



In [ ]:
from copy import deepcopy

def get_activated_model(model, plus_prompt, minus_prompt, plus_coeff, minus_coeff, layer):
  activation_model = deepcopy(model)
  orig = model.transformer.h[layer].forward

  activation = plus_prompt
  acts = dict()

  def new_act(hidden_states, layer_past=None, attention_mask=None, head_mask=None, use_cache=False, output_attentions=False):
    val = orig(hidden_states, layer_past, attention_mask, head_mask, use_cache, output_attentions)
    acts[activation] = val
    return val

  activation_model.transformer.h[layer].forward = new_act
  activation = plus_prompt

  input_ids = tokenizer.encode(activation, return_tensors="pt").to("cuda")
  activation_model(input_ids)


  activation = minus_prompt
  input_ids = tokenizer.encode(activation, return_tensors="pt").to("cuda")
  activation_model(input_ids)

  def add_acts(hidden_states, layer_past=None, attention_mask=None, head_mask=None, use_cache=False, output_attentions=False):
    val = orig(hidden_states, layer_past, attention_mask, head_mask, use_cache, output_attentions)


    a = val[0] + plus_coeff* acts[plus_prompt][0] - minus_coeff*acts[minus_prompt][0]
    #x = val[1][0] +  acts['boy'][1][0] - acts['girl'][1][0]
    #y = val[1][1] + acts['boy'][1][1] -  acts['girl'][1][1]
    return (a, val[1])

  activated_model = deepcopy(model)
  activated_model.transformer.h[layer].forward = add_acts


  return activated_model

In [ ]:
model = AutoModelForCausalLM.from_pretrained("roneneldan/TinyStories-33M").to("cuda")

activated_model = get_activated_model(model, 'girl', 'boy', 1, 0.7, 1)

input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda")



output = activated_model.generate(input_ids, max_length=500, do_sample=True)

output_text = tokenizer.decode(output[0], skip_special_tokens=True)
print()
print(output_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Once upon a time there was a brave and strong prince named Lucy. She was going to the princesseses was.

The princess was, and she wasn't. She always did her best to get organized. 
"Let's make it to look her dress. her own dress, she put this dress just her. 

"The words said, the dress made her feel embarrassed.

But then, she started to make her. She was too late.

"I'm so angry and not shy. She would really want to make the dress a pretty," she said.

So, she decided to set her own. 

"Hanna was, Anna. I want to feel more brave, " she wanted to make sure it should go away. That made, the girl's day.

Anna was angry her and she was brave, but not too angry. It was she wasn't a pretty dress for her.

"girl," she said.

"The pink dress is so pretty, she wanted to be brave and pretty." she said, and then she got her dress.

The dress and the doll smiled and put her little girl. She was all her. She was so happy.
 

"I found a pretty, and they all put!" She said.

Just then, she put on

In [ ]:
#model = AutoModelForCausalLM.from_pretrained("roneneldan/TinyStories-33M").to("cuda")

activated_model = get_activated_model(model, 'boy', 'girl', 1.1, 0.5, 1)

input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda")



output = activated_model.generate(input_ids, max_length=500, do_sample=True)

output_text = tokenizer.decode(output[0], skip_special_tokens=True)
print()
print(output_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Once upon a time there was a boy named Sammy. In was a big old man walking by. He was old and grilling. He saw it was his friend, saying to him and asking "wow! here was a ladder!'. 

one said, 'Why is you was so old, man.
 His friend! Toothy is so tall? To his wondered about them, a 3 year old down was people's feet in search of Toffee'.

His lesson to learn what was walking is. He was trying to find his mom's foot was still walking away from the people's feet. His legs were so clean! One.



In [ ]:
activated_model = get_activated_model(model, 'walk', 'sit', 1, 1, 1)

input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda")



output = activated_model.generate(input_ids, max_length=500, do_sample=True)

output_text = tokenizer.decode(output[0], skip_special_tokens=True)
print()
print(output_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Once upon a time there was an ordinary house. As he was walking he saw a big mountain. He was so excited! He didn't forget how much forgot! 

Tom was running to the left. As he stepped in, he saw a large, dark vehicle. He was so surprised, he forgot the mirror! He turned and soon the left and didn Oh.

On a full! He was so sad. He was leaving without his. He was leaving without a.

He was so speeding. He was he drove by in speeding. He was soon in the way. He saw a pass before the crossed he forgot. He saw a big, school school. He was turninged. By the, the he he was too. He forgot he could. And he got to the left and he crashed in the car. He smashed into a Oh! There was! And he forgot he didn't the other. He was so in theening he locked in the pasted. 

In the mine. As he did, he saw. He saw a very old in full traffic. He was so speeding. He was speeding. He crashed into a bright yellow. He was on the,".

When he turned drove, he could not. He was speeding. He drove faster and, soon

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.3 MB/s eta 0:00:00


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-1_5", trust_remote_code=True
).to('cuda')

tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5")
model.eval()

MixFormerSequentialForCausalLM(
  (layers): Sequential(
    (0): Embedding(
      (wte): Embedding(51200, 2048)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (1): ParallelBlock(
      (ln): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (resid_dropout): Dropout(p=0.0, inplace=False)
      (mixer): MHA(
        (rotary_emb): RotaryEmbedding()
        (Wqkv): Linear(in_features=2048, out_features=6144, bias=True)
        (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
        (inner_attn): SelfAttention(
          (drop): Dropout(p=0.0, inplace=False)
        )
        (inner_cross_attn): CrossAttention(
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
      (mlp): MLP(
        (fc1): Linear(in_features=2048, out_features=8192, bias=True)
        (fc2): Linear(in_features=8192, out_features=2048, bias=True)
        (act): NewGELUActivation()
      )
    )
    (2): ParallelBlock(
      (ln): LayerNorm((2048,), eps=1e-05, elementwis

In [ ]:
prompt = "Write a movie review"

input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda")

output = model.generate(input_ids, max_length=500, do_sample=True)

output_text = tokenizer.decode(output[0], skip_special_tokens=True)
print()
print(output_text)


Write a movie review for "Love and the Dragon" based on your personal experience watching the movie.

Exercise 4: Discuss the impact of the movie on its audience both visually and emotionally.

Exercise 5: Create a poster promoting a movie of your choice, using persuasive language to attract viewers.

Answer:

Exercise 1: Three popular genres of movies are action, comedy, and romance. Action movies often involve thrilling sequences and stunts, comedy movies aim to make the audience laugh with funny jokes, and romance movies focus on love stories and relationships.

Exercise 2: Your favorite movie from the given genres could be an action movie like "Mission Impossible" with thrilling sequences and high-speed chases, a comedy movie like "Bridesmaids" that uses slapstick humor and witty dialogue, or a romance movie like "The Notebook" that follows the central theme of love and relationships.

Exercise 3: Watching "Love and the Dragon" made me feel a mix of emotions. There were intense ac

In [ ]:
tokenizer.pad_token = tokenizer.eos_token


In [ ]:
from copy import deepcopy


def get_activated_model(model, plus_prompt, minus_prompt, plus_coeff, minus_coeff, layer):
  activation_model = deepcopy(model)
  orig = model.layers[layer].forward

  tlen = lambda prompt: len(tokenizer.encode(prompt))
  pad_right = lambda prompt, length: prompt + "<|endoftext|>" * (length - tlen(prompt))
  l = max(tlen(plus_prompt), tlen(minus_prompt))
  plus_prompt, minus_prompt = pad_right(plus_prompt, l), pad_right(minus_prompt, l)

  print(f"'{plus_prompt}'", f"'{minus_prompt}'")

  activation = plus_prompt
  acts = dict()

  def new_act(*args, **kwargds):
    val = orig(*args, **kwargds)
    acts[activation] = val
    return val

  activation_model.layers[layer].forward = new_act
  activation = plus_prompt

  input_ids = tokenizer.encode(activation, return_tensors="pt").to("cuda")
  activation_model(input_ids)


  torch.cuda.empty_cache()


  activation = minus_prompt
  input_ids = tokenizer.encode(activation, return_tensors="pt").to("cuda")
  activation_model(input_ids)

  del activation_model
  torch.cuda.empty_cache()


  def add_acts(*args, **kwargds):
    val = orig(*args, **kwargds)


    a = val + plus_coeff* acts[plus_prompt] - minus_coeff*acts[minus_prompt]
    #x = val[1][0] +  acts['boy'][1][0] - acts['girl'][1][0]
    #y = val[1][1] + acts['boy'][1][1] -  acts['girl'][1][1]
    return a

  activated_model = deepcopy(model)
  activated_model.layers[layer].forward = add_acts


  return activated_model

In [ ]:

activated_model = get_activated_model(model, 'bad', 'good', 1, 1, 5)


'bad' 'good'


In [ ]:
prompt = "Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestions? Bob:"
input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda")



output = activated_model.generate(input_ids, max_length=50, do_sample=True)

output_text = tokenizer.decode(output[0], skip_special_tokens=True)
print()
print(output_text)


Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestions? Bob: So I chose the new headphones since they weren't good.


John had a huge ego and tried to assert their newfound confidence
